#### Importing all libraries

In [ ]:
import pandas as PD
import seaborn as SNS
import matplotlib.pyplot as PLT
import numpy as NP
from numpy import sum

In [ ]:
custom_colours = ['#74a09e','#98e2c6','#f2a553','#c14953']
SNS.set_theme(style="whitegrid",palette=custom_colours)

#### Loading the dataset

In [ ]:
raw_data = PD.read_csv("50_Startups.csv", sep=";")
data = raw_data.copy()

#### Initial data exploration

In [ ]:
data.info()

This dataset has **50** entries in **5** columns. It has **no missing data**. **4** columns are **numerical** and **1** column is **categorical**.

In [ ]:
data.head()

Columns are ordered as follows:
- **3** columns detailing the **budget** each startup grants to each of their departments: **R&D**, **Administration** and **Marketing**.
- **1** column detailing the **location** where the startup supposedly operates from.
- **1** column detailing the **profit** each startup has made over an **unknown timespan**.

In [ ]:
print(data["villes"].value_counts())
PLT.pie(data["villes"].value_counts(),labels=data["villes"].unique(),autopct='%1.1f%%')
PLT.show()

We see here that we can order the dataset by the **'villes'** column since it is balanced around those values, with an even spread between all three of the cities found in that column : **Strasbourg**, **Paris** and **Lyon**.

In [ ]:
SNS.catplot(data=data,kind="bar",x="villes",y="Profit")
PLT.show()

We can see from this graph that, at face value, the "best" city to operate from is **Lyon** since it boasts both the highest **average profit** and **maximum profit**.

In [ ]:
fig, ax = PLT.subplots(1,3,figsize=(18,9),sharey=True)
SNS.boxenplot(data=data,y="R&D",ax=ax[0])
ax[0].set_xlabel("R&D")
ax[0].set_ylabel("")
SNS.boxenplot(data=data,y="Administration",ax=ax[1])
ax[1].set_xlabel("Administration")
ax[1].set_ylabel("")
SNS.boxenplot(data=data,y="Marketing",ax=ax[2])
ax[2].set_xlabel("Marketing")
ax[2].set_ylabel("")
PLT.show()

By far, the highest budget allocation goes to the marketing department, it also demonstrates that there is a large variety of investment strategies for marketing whereas the budget for administration seems to have been figured out by most, as if they all shared a similar template; the same cannot be said for R&D but its overall budget remains the lowest across the board.

In [ ]:
data.corr()

At first glance, we can see a strong correlation between R&D and Profit, a significant correlation between Marketing and Profit and almost none between Administration and Profit. This is not unexpected since the administration department cannot be expected to have as direct an impact on profit.

#### Study of correlations

In [ ]:
SNS.heatmap(data.corr())
PLT.show()

In [ ]:
SNS.jointplot(x="R&D",y="Profit",kind="reg",data=data)
PLT.show()

In [ ]:
SNS.jointplot(x="Administration",y="Profit",kind="reg",data=data)
PLT.show()

In [ ]:
SNS.jointplot(x="Marketing",y="Profit",kind="reg",data=data)
PLT.show()

In [ ]:
data["Budget"] = data["R&D"]+data["Administration"]+data["Marketing"]
data.head()

We decide to sum up all bugdets together in a new 'Budget' column to see the correlation between the overall budget of a startup and their profit. We expect a loss in precision in our model.

In [ ]:
SNS.jointplot(x="Budget",y="Profit",kind="reg",data=data)
PLT.show()

Correlation with new features

In [ ]:
data.corr()

Expectedly, the idea that you need to 'spend money to make money' seems to be holding true, with an almost normal distribution of our values.

#### Linear Regression Model(s)

##### Importing all machine learning libraries

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

##### Choosing our features and target

In [ ]:
LR = LinearRegression()
X = data.loc[:,data.columns != 'Profit'].copy()
ohe_encoder = OneHotEncoder(sparse=False, drop='first')
encoded_villes = ohe_encoder.fit_transform(PD.DataFrame(data.loc[:,'villes']))
encoded_villes_df = PD.DataFrame(encoded_villes, columns=ohe_encoder.get_feature_names_out())
X = PD.concat([X,encoded_villes_df],axis=1).drop('villes',axis=1)
#X.drop("villes",axis=1,inplace=True)
y = data['Profit'].copy()
X.head()

We have decided to encode and use the "villes" feature.

Creating the training and testing datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=666)

##### Reinventing the wheel

In [ ]:
def R2_AdjR2(y,f,nb_features):
    p = nb_features
    n = len(y)
    SSres = 0
    for i in range(n):
        ei = y[i] - f[i]
        SSres += ei**2
    y_ = (1/n)*sum(y)
    SStot = 0
    for i in range(n):
        SStot += (y[i]-y_)**2
    R2 = 1 - (SSres/SStot)
    dfe = n-p-1
    dft = n-1
    adj_R2 = 1 - (SSres/dfe)/(SStot/dft) # adj_R2 = 1 - (1-R2)*(n-1)/(n-p-1)
    return R2, adj_R2

##### Training and scoring the Linear Regression model

In [ ]:
LR.fit(X_train,y_train)
predictions = LR.predict(X_test)
res_df = PD.DataFrame(dict(R_et_D=X_test["R&D"],Administration=X_test['Administration'],Marketing=X_test['Marketing'], Budget=X_test['Budget'],Paris=X_test['villes_Paris'], Strasbourg=X_test['villes_Strasbourg'],y_true=y_test,y_pred=predictions))
Y = NP.array(y_test)
f = NP.array(predictions)
R2, adj_R2 = R2_AdjR2(Y,f,LR.n_features_in_)
print(f"R²:\n{R2}\n")
print(f"Adjusted R²:\n{adj_R2}\n")
print(f"Explained Variance Score:\n{explained_variance_score(y_test,predictions)}\n")
print(f"Coefficients:\n{LR.coef_}\n")
print(f"MAE:\n{mean_absolute_error(y_test, predictions)}\n")
print(f"MSE:\n{mean_squared_error(y_test, predictions)}\n")
print(f"RMSE:\n{mean_squared_error(y_test, predictions)**(1/2)}")

##### Comparing our predictions, for each feature, with the true values *(pink is prediction, green is true)*

In [ ]:
fig, ax = PLT.subplots(1,4,figsize=(20,5), sharey=True)
SNS.regplot(x="R_et_D",y="y_pred",data=res_df, ax=ax[0], color="#8F3975")
SNS.regplot(x="R_et_D",y="y_true",data=res_df, ax=ax[0], color="#398F53")
ax[0].set_ylabel("y_pred AND y_true")

SNS.regplot(x="Administration",y="y_pred",data=res_df, ax=ax[1], color="#8F3975")
SNS.regplot(x="Administration",y="y_true",data=res_df, ax=ax[1], color="#398F53")
ax[1].set_ylabel("")

SNS.regplot(x="Marketing",y="y_pred",data=res_df, ax=ax[2], color="#8F3975")
SNS.regplot(x="Marketing",y="y_true",data=res_df, ax=ax[2], color="#398F53")
ax[2].set_ylabel("")

SNS.regplot(x="Budget",y="y_pred",data=res_df, ax=ax[3], color="#8F3975")
SNS.regplot(x="Budget",y="y_true",data=res_df, ax=ax[3], color="#398F53")
ax[3].set_ylabel("")
PLT.show()

These graphs show that the 'Administration' feature could be introducing some bias. 
We will confirm our intuition with several tests.

RFE

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

estimator_svr = SVR(kernel="linear")
selector_svr = RFE(estimator_svr)
selector_svr = selector_svr.fit(X, y)

print(f'SVR:')
print(f'{selector_svr.support_}')
print(f'{selector_svr.ranking_}\n')

estimator_tree= DecisionTreeRegressor()
selector_tree = RFE(estimator_tree)
selector_tree = selector_tree.fit(X, y)

print(f'Tree:')
print(f'{selector_tree.support_}')
print(f'{selector_tree.ranking_}')

F-ANOVA

In [ ]:
from sklearn.feature_selection import f_regression
f_statistic, p_values = f_regression(X,y)
print(f"F values:\n{f_statistic}\n")
print(f"P values:\n{p_values}")

K-BEST

In [ ]:
from sklearn.feature_selection import SelectKBest

X_kbest = SelectKBest(f_regression, k=4).fit(X, y)
print(X_kbest.get_feature_names_out())

In [ ]:
X.drop(columns=['villes_Paris','villes_Strasbourg']).corr()

After a few tests during our feature selection, we can remove the encoded columns, thus confirming our initial intuition that we should not be using the 'villes' column.

##### Training and scoring the Linear Regression model with the new set of features

In [ ]:
LR_A = LinearRegression()
Xa = X.drop(columns=['villes_Paris','villes_Strasbourg'])
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa,y,test_size=.3,random_state=666)
LR_A.fit(Xa_train,ya_train)
A_predictions = LR_A.predict(Xa_test)
Y = NP.array(ya_test)
f = NP.array(A_predictions)
R2, adj_R2 = R2_AdjR2(Y,f,LR_A.n_features_in_)
print(f"R²:\n{R2}\n")
print(f"Adjusted R²:\n{adj_R2}\n")
print(f"Explained Variance Score:\n{explained_variance_score(ya_test,A_predictions)}\n")
print(f"Coefficients:\n{LR_A.coef_}")
print(f"MAE:\n{mean_absolute_error(ya_test, A_predictions)}\n")
print(f"MSE:\n{mean_squared_error(ya_test, A_predictions)}\n")
print(f"RMSE:\n{mean_squared_error(ya_test, A_predictions)**(1/2)}")

When compared with the previous **adjusted R² score** *(0.8622366684019128)* we see an improvement of ~3%; however, we have a feeling we can improve our model further.


##### Second round of features selection

In [ ]:
X_kbest = SelectKBest(f_regression, k=2).fit(Xa, y)
print(X_kbest.get_feature_names_out())

##### Training the new model

In [ ]:
LR_B = LinearRegression()
Xb = Xa.drop(columns=['Marketing','Administration'])
Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb,y,test_size=.3,random_state=666)
LR_B.fit(Xb_train,yb_train)
B_predictions = LR_B.predict(Xb_test)
Y = NP.array(yb_test)
f = NP.array(B_predictions)
R2, adj_R2 = R2_AdjR2(Y,f,LR_B.n_features_in_)
print(f"R²:\n{R2}\n")
print(f"Adjusted R²:\n{adj_R2}\n")
print(f"Explained Variance Score:\n{explained_variance_score(yb_test,B_predictions)}\n")
print(f"Coefficients:\n{LR_A.coef_}")
print(f"MAE:\n{mean_absolute_error(yb_test, B_predictions)}\n")
print(f"MSE:\n{mean_squared_error(yb_test, B_predictions)}\n")
print(f"RMSE:\n{mean_squared_error(yb_test, B_predictions)**(1/2)}")

When compared with the previous **adjusted R² score** *(0.8986509064888631)* we see an improvement of ~3% again; however, with the model down to only two features, we can't go any further.

The new model is **objectively** better, thus we'll use it for the prediction.

In [ ]:
R_n_D_budget = 65135
Marketing_budget = 353000
Administration_budget = 150000
Budget_budget = R_n_D_budget + Marketing_budget + Administration_budget

big_dict = {"R&D":R_n_D_budget,"Budget":Budget_budget}
x_try = PD.DataFrame(big_dict, index=[0])
x_try.head()

In [ ]:
try_predict = LR_B.predict(x_try)
print(try_predict)

### Rapport

- **Nom:** Victorino, Thiberino
- **Date:** Oui.
- **Cible client:** Non. 

**Contexte:**
> "La BPI France dispose d'un fond d'investissement qu'elle voudrait utiliser pour investir dans les start'up de demain les plus
prometteuses. Seulement elle ne sait pas comment les sélectionner. 
Faut-il investir dans celles qui dépensent le plus en marketing? en recherche et développement? dans quelles villes les startups semblent mieux opérer? Elle fait donc appel à vous pour y voir plus clair..."

**Objectif:**
> "Vous devez donc concevoir un modèle de régression linéaire multiple qui permettra à la BPI d'une part de sélectionner les 5
start'up les plus prometteuses et d'autre part de déterminer dans quel(s) secteur(s) il serait le plus judicieux de répartir les
budgets de dépenses."

*Faut -il investir dans les entreprises qui dépensent le plus en marketing ou en recherche et développement?*

In [ ]:
data_1 = data.copy()
data_1.drop("Administration",axis=1,inplace=True)
data_1.drop("villes",axis=1,inplace=True)
data_1.drop("Budget",axis=1,inplace=True)
data_1.sort_values(by="Profit",ascending=False)
print(f"Max Marketing:\n{data_1['Marketing'].max()}")
print(f"Max R&D:\n{data_1['R&D'].max()}")
data_1.head()

Au vu de la forte correlation entre ces deux features, investir dans l'un ou l'autre ménera à l'augmentation des profits.

*Dans quelles villes les startups semblent elles mieux opérer?*

In [ ]:
SNS.catplot(data=data,kind="bar",x="villes",y="Profit")
PLT.show()

Lyon est plus profitable que Paris qui est plus profitable que Strasbourg.

En conclusion, si les postes de dépenses sont fortement corrélés avec le profit, il est presque assuré que le profit augmente si le budget augmente *(dans des conditions idéales)*. Investir dans le R&D offre un meilleur retour sur investissement.